## Латыпов Ильяс Дамирович (DS-22)
## Домашнее задание №3.

В этом задании вам необходимо будет:

1. Реализовать формулу подсчета длительности теста, сравнить ее с онлайн калькуляторами (например https://mindbox.ru/tools/ab-test-calculator/ ). При сравнении оценить мощность критерия при указанном изменении и рассчитанном количестве наблюдений в выборке. 

2. Реализовать метод линеаризации. Проверить для него корректность и мощность. Мощность должна быть больше, чем просто на обычных значениях конверсии пользователей.

3. Реализовать метод CUPED. Проверить для него корректность и мощность. Данные на этапе до A/B тесте необходимо сгенерировать один раз, далее синтетически генерировать только часть, связанную с проведением A/B-теста.


In [1]:
import numpy as np
import scipy.stats as sps
import pandas as pd
import plotly.graph_objs as go

In [2]:
def power(distribution, alpha=0.05):
    return len(distribution[distribution < alpha]) / len(distribution)

def samples_dist(loc, scale, loc_effect, scale_effect, size=100, exp_dist=False): #normal dist - default
    if exp_dist:
        sample1 = sps.expon.rvs(loc=loc, scale=scale, size=size)
        sample2 = sps.expon.rvs(loc=loc + loc_effect, scale=scale + scale_effect, size=size)
    else:
        sample1 = sps.norm.rvs(loc=loc, scale=scale, size=size)
        sample2 = sps.norm.rvs(loc=loc + loc_effect, scale=scale + scale_effect, size=size)
        
    return sample1, sample2

def p_values_dist(loc, scale, loc_effect, scale_effect, size=100, n_samples=100, exp_dist=False): #normal dist - default
    p_values = []
    p_values_m = []
    
    for _ in range(n_samples):
        sample1, sample2 = samples_dist(loc, scale, loc_effect, scale_effect, size=size, exp_dist=False)
        p_values.append(sps.ttest_ind(sample1, sample2, equal_var=False).pvalue)
        p_values_m.append(sps.mannwhitneyu(sample1, sample2).pvalue)

    return np.array(p_values), np.array(p_values_m)

def samples_dist_bernoulli(p, p_effect, size=100):
    sample1 = sps.bernoulli.rvs(p, size=size)
    sample2 = sps.bernoulli.rvs(p + p_effect, size=size)
        
    return sample1, sample2

def p_values_dist_bernoulli(p, p_effect, size=100, n_samples=100):
    p_values = []
    p_values_m = []
    
    for _ in range(n_samples):
        sample1, sample2 = samples_dist_bernoulli(p, p_effect, size=size)
        p_values.append(sps.ttest_ind(sample1, sample2, equal_var=False).pvalue)
        p_values_m.append(sps.mannwhitneyu(sample1, sample2).pvalue)

    return np.array(p_values), np.array(p_values_m)

def Q_Q_plot(p_values, name, n_samples=1000):
    probs = []
    x = [i / n_samples for i in range(n_samples + 1)]
    for i in range(n_samples + 1):
        alpha_step = i / n_samples
        probs.append(p_values[p_values < alpha_step].shape[0] / p_values.shape[0])
    fig = go.Figure(
        [
            go.Scatter(x=x, y=probs, mode="markers", name="p_value"),
            go.Scatter(x=x, y=x, mode="lines", name="uniform"),
        ]
    )
    fig.update_layout(height=550, width=550, title=f"Q-Q plot for {name}")
    fig.show()

### 1. Длительность теста

In [3]:
def duration(k, delta_effect, sigma_1, sigma_2, alpha=0.05, beta=0.2):
    z = sps.norm.ppf(1 - alpha/2) + sps.norm.ppf(1-beta)
    n = (k+1) * z ** 2 * (sigma_1 ** 2 + sigma_2 **2 / k) / (delta_effect ** 2)
    return n

def sigma(p):
    return np.sqrt(p * (1 - p))

In [4]:
# Проведем серию экспериментов
alpha = 0.05
n_samples = 1000
data = []
cols = ["A.distribution", "A.mean", "A.std", "A.mean_effect", "A.std_effect", "Ожидаемая Мощность",
        "Мощность t-test",  "Мощность Mann-Whitney", "duration"]
distr_names =["norm", "exp"]
powers = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
    
for pw in powers:
    for exp_dist in [False, True]: # norm and exp distribution
        m0, v0 = np.random.randint(10, 20), np.random.randint(10, 25)
        m0_effect = np.random.randint(5, 15)
        v0_effect = np.random.randint(5, 25)
        dur = duration(1, m0_effect, v0, v0 + v0_effect, alpha=alpha, beta=1 - pw)

        p_values, p_values_m = p_values_dist(m0, v0 , m0_effect, v0_effect, int(dur // 2), n_samples, exp_dist)
        data.append([distr_names[exp_dist], m0, v0, m0_effect, v0_effect, pw,
                     power(p_values), power(p_values_m), int(dur)])

print("alpha = ", alpha)
pd.DataFrame(data, columns=cols)

alpha =  0.05


,A.distribution,A.mean,A.std,A.mean_effect,A.std_effect,Ожидаемая Мощность,Мощность t-test,Мощность Mann-Whitney,duration
0,norm,10,24,14,10,0.9,0.909,0.894,185
1,exp,18,11,14,17,0.9,0.900,0.869,97
2,norm,14,17,6,5,0.8,0.790,0.772,337
3,exp,11,20,5,16,0.8,0.797,0.766,1064
4,norm,12,11,7,15,0.7,0.664,0.638,200
5,exp,15,17,13,17,0.7,0.690,0.651,105
6,norm,10,11,8,7,0.6,0.602,0.577,68
7,exp,10,21,10,17,0.6,0.608,0.585,184
8,norm,13,14,5,20,0.5,0.467,0.461,415
9,exp,19,17,14,8,0.5,0.457,0.434,35


In [5]:
# Проведем серию экспериментов для конверсии
alpha = 0.05
n_samples = 1000
probs = [0.2, 0.3]
powers = [0.8, 0.7]
delta_effects = [0.05, 0.1]
data = []
cols = ["p", "effect", "Ожидаемая Мощность",  
        "Мощность t-test", "Мощность Mann-Whitney", "Duration"]

for p in probs:
    for pw in powers:
        for effect in delta_effects:
            dur = duration(1, effect, sigma(p), sigma(p + effect), alpha, beta = 1 - pw)            
            p_values, p_values_m = p_values_dist_bernoulli(p, effect, size=int(dur // 2), n_samples=n_samples)
            data.append([p, effect, pw, power(p_values), power(p_values_m), int(dur)])

print("alpha = ", alpha)
df = pd.DataFrame(data, columns=cols)
df

alpha =  0.05


,p,effect,Ожидаемая Мощность,Мощность t-test,Мощность Mann-Whitney,Duration
0,0.2,0.05,0.8,0.803,0.803,2181
1,0.2,0.10,0.8,0.790,0.790,580
2,0.2,0.05,0.7,0.700,0.700,1715
3,0.2,0.10,0.7,0.686,0.686,456
4,0.3,0.05,0.8,0.799,0.799,2747
5,0.3,0.10,0.8,0.808,0.808,706
6,0.3,0.05,0.7,0.706,0.706,2160
7,0.3,0.10,0.7,0.702,0.702,555


In [6]:
df.iloc[0:1]

,p,effect,Ожидаемая Мощность,Мощность t-test,Мощность Mann-Whitney,Duration
0,0.2,0.05,0.8,0.803,0.803,2181


![HW3_pic1.png](HW3_pic1.png)

In [7]:
df.iloc[7:8]

,p,effect,Ожидаемая Мощность,Мощность t-test,Мощность Mann-Whitney,Duration
7,0.3,0.1,0.7,0.702,0.702,555


![HW3_pic2.png](HW3_pic2.png)

### 2. Линеаризация

In [11]:
def p_values_func(p=0.05, effect=0, loc=100, scale=100, n_exp=1000):   
    p_values = []
    p_values_lin = []
    for _ in range(n_exp):
        records = []
        for i in range(100):
            n_views = int(sps.expon.rvs(loc=loc, scale=scale))
            clicks = sps.bernoulli.rvs(p=p, size=n_views)
            records.append([n_views, np.sum(clicks), np.sum(clicks)/ n_views, "A"])
        for i in range(100):
            n_views = int(sps.expon.rvs(loc=loc, scale=scale))
            clicks = sps.bernoulli.rvs(p=p+effect, size=n_views)
            records.append([n_views, np.sum(clicks), np.sum(clicks)/ n_views, "B"])
        df_data = pd.DataFrame(records, columns=["views", "clicks", "cr", "group"])
        cr_A = df_data[df_data["group"] == "A"]["clicks"].sum() / df_data[df_data["group"] == "A"]["views"].sum()
        df_data["cr_lin"] = df_data["clicks"] - cr_A * df_data["views"]

        x_a = df_data[df_data["group"] == "A"]["cr"]
        x_b = df_data[df_data["group"] == "B"]["cr"]
        p_value = sps.ttest_ind(x_a, x_b).pvalue
        p_values.append(p_value)

        x_a_lin = df_data[df_data["group"] == "A"]["cr_lin"]
        x_b_lin = df_data[df_data["group"] == "B"]["cr_lin"]
        p_value_lin = sps.ttest_ind(x_a_lin, x_b_lin).pvalue
        p_values_lin.append(p_value_lin)
    
    return np.array(p_values), np.array(p_values_lin)

In [34]:
%%time
# Проведем серию экспериментов
n_samples = 1000
probs = [0.05, 0.15]
delta_effects = [0.005, 0.01]
data = []
cols = ["p (clicks)", "effect (clicks)", "mean (n_views)", "std (n_views)", "Мощность", "Мощность линейная"]

for p in probs:
    for effect in delta_effects:
        m0, v0 = np.random.randint(100, 120), np.random.randint(110, 150)
        p_values, p_values_lin = p_values_func(p, effect, m0, v0, n_samples)
        data.append([p, effect, m0, v0, power(p_values), power(p_values_lin)])

pd.DataFrame(data, columns=cols)

Wall time: 1min 47s


,p (clicks),effect (clicks),mean (n_views),std (n_views),Мощность,Мощность линейная
0,0.05,0.005,113,132,0.627,0.706
1,0.05,0.010,111,112,0.988,0.996
2,0.15,0.005,119,139,0.290,0.337
3,0.15,0.010,115,132,0.792,0.850


In [30]:
# Проверим корректность
p_values, p_values_lin = p_values_func(p=0.05, effect=0, loc=100, scale=100, n_exp=n_samples)
Q_Q_plot(p_values, "base")

In [31]:
Q_Q_plot(p_values_lin, "linear")

### Screens for GitHub

![HW3_pic3.png](HW3_pic3.png)

![HW3_pic4.png](HW3_pic4.png)

### 3. CUPED

In [41]:
def p_values_cuped_func(pre_exp, loc=1.0, loc_effect=0, scale=0.1, size = 1000, n_exp=1000):   
    p_values = []
    p_values_cuped = []

    for _ in range(n_exp):

        df_A = pd.DataFrame()
        df_A["user"] = [f"A_{x:5}" for x in range(size)]
        df_A["pre_exp"] = pre_exp
        df_A["payments"] =  sps.norm.rvs(loc=loc, scale=scale, size=size) * df_A["pre_exp"]

        df_B = pd.DataFrame()
        df_B["pre_exp"] = pre_exp
        df_B["user"] = [f"B_{x:5}" for x in range(size)]
        df_B["payments"] = sps.norm.rvs(loc=loc+loc_effect, scale=scale, size=size) * df_B["pre_exp"]

        p_values.append(sps.ttest_ind(df_A["payments"], df_B["payments"]).pvalue)

        x_a = df_A["pre_exp"]
        x_b = df_B["pre_exp"]
        y_a = df_A["payments"]
        y_b = df_B["payments"]
        theta = np.cov(x_a, y_a)[0,1] / np.std(x_a)**2

        df_A["payments_cuped"] = df_A["payments"] - theta * df_A["pre_exp"]
        df_B["payments_cuped"] = df_B["payments"] - theta * df_B["pre_exp"]

        p_values_cuped.append(sps.ttest_ind(df_A["payments_cuped"], df_B["payments_cuped"]).pvalue)
    
    return np.array(p_values), np.array(p_values_cuped)

In [70]:
%%time
# Проведем серию экспериментов
size = 1000
n_samples = 1000
pre_exp = sps.norm.rvs(loc=100, scale=20, size=size)

locs = [1.0, 2.0]
loc_effects = [0.005, 0.01, 0.05, 0.1]
scales = [0.1, 0.2, 0.4]
data = []
cols = ["mean", "effect", "std", "Мощность", "Мощность cuped"]

for loc in locs:
    for effect in loc_effects:
        for scale in scales:
            p_values, p_values_cuped = p_values_cuped_func(pre_exp, loc, effect, scale, size, n_samples)
            data.append([loc, effect, scale, power(p_values), power(p_values_cuped)])

pd.DataFrame(data, columns=cols)

Wall time: 2min 36s


,mean,effect,std,Мощность,Мощность cuped
0,1.0,0.005,0.1,0.001,0.202
1,1.0,0.005,0.2,0.014,0.089
2,1.0,0.005,0.4,0.044,0.070
3,1.0,0.010,0.1,0.017,0.584
4,1.0,0.010,0.2,0.046,0.206
5,1.0,0.010,0.4,0.063,0.090
6,1.0,0.050,0.1,1.000,1.000
7,1.0,0.050,0.2,0.996,0.999
8,1.0,0.050,0.4,0.687,0.760
9,1.0,0.100,0.1,1.000,1.000


In [67]:
# Проверим корректность
size = 1000
n_samples = 1000
pre_exp = sps.norm.rvs(loc=100, scale=20, size=size)
p_values, p_values_cuped = p_values_cuped_func(pre_exp, loc=1.0, loc_effect=0, scale=0.1, size = size, n_exp=n_samples)

In [68]:
Q_Q_plot(p_values_cuped, "Cuped")

### Screens for GitHub

![HW3_pic5.png](HW3_pic5.png)